In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
spark.version

In [ ]:
!ls -lh fhvhv_tripdata_2021-06.csv

In [ ]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv')

df = df.repartition(24)

df.write.parquet('data/pq/fhvhv/2021/06/', compression="gzip")

In [ ]:
df = spark.read.parquet('data/pq/fhvhv/2021/06/')

**Q3**: How many taxi trips were started on June 15th?

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

In [ ]:
df.registerTempTable('fhvhv')

In [ ]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv
WHERE
    to_date(pickup_datetime) = '2021-06-15';
""").show()

**Q4**: How long is the longest trip in the dataset?

In [ ]:
spark.sql("""SELECT
        to_date(pickup_datetime) AS pickup_date,
        MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
        FROM
            fhvhv
        GROUP BY
            1
        ORDER BY
             2 DESC LIMIT 10;""").show()

**Q6**: What is the name of the most frequent pickup location zone?

In [ ]:
In [41]: spark.sql("""SELECT
             PULocationID,
             COUNT(*) AS qty
         FROM
             fhvhv
         GROUP BY
             1
         ORDER BY qty DESC
         LIMIT 10;""").show()